In [11]:
### Importerer nødvendige biblioteker
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from poweranalysis.weather.frost_client import FrostDataRetriever
from poweranalysis.config import Config
from poweranalysis.models.hdd import HDDModel

In [12]:
### Laster inn data, vil du se mer; ref build_dataset.py
df_daily = pd.read_parquet(Config.out_daily)
df_hourly = pd.read_parquet(Config.out_hourly)

In [13]:
df_daily.head()

,Forbruk (kWh),Pris (øre/kWh),Nettleie (øre/kWh),air_temperature,relative_humidity,wind_speed,wind_from_direction,sum(precipitation_amount PT1H),hour,dayofweek,is_weekend,month
Tidspunkt,,,,,,,,,,,,
2025-08-01,3.576,21.877667,45.286667,20.273913,81.521739,2.039130,172.652174,0.0000,11.5,4.0,0.0,8.0
2025-08-02,3.630,16.449558,45.286667,18.425000,87.916667,1.300000,256.083333,0.0000,11.5,5.0,1.0,8.0
2025-08-03,3.105,11.334342,45.286667,19.941667,79.791667,2.033333,182.125000,0.0000,11.5,6.0,1.0,8.0
2025-08-04,3.705,17.420217,45.286667,17.287500,85.625000,1.550000,157.500000,0.0500,11.5,0.0,0.0,8.0
2025-08-05,3.520,6.222729,45.286667,16.925000,66.916667,4.000000,194.875000,0.0875,11.5,1.0,0.0,8.0
